In [ ]:
import yfinance as yf
import pandas as pd
from tabulate import tabulate

# Stock list with all user-requested symbols
stock_list = [
    'BRK-B','MELI','IBKR','AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'TSLA',
    'ASTS', 'HOOD', 'AMD', 'TSM', 'UPST', 'NU', 'TEM',
    'ADBE', 'ADYEY', 'ASML', 'CELH', 'CPRT', 'CRWD', 'DDOG', 'DUOL', 'FRPH',
    'HIMS', 'MSTR', 'PAYC', 'ROKU', 'TTD', 'TXN'
]

# Download SPY for Mansfield RS benchmark
spy = yf.download('SPY', period='5y', interval='1wk', auto_adjust=True, progress=False)
spy['SPY_Return'] = spy['Close'].pct_change()
spy = spy[['SPY_Return']]

all_results = []

def analyze_stock(ticker):
    try:
        df = yf.download(ticker, period='5y', interval='1wk', auto_adjust=True, progress=False)

        if df.empty or 'Close' not in df.columns:
            print(f"❌ {ticker}: Missing or empty data.")
            return {'Ticker': ticker, 'Buy': 'No Data'}

        df['50MA'] = df['Close'].rolling(window=50).mean()
        df['200MA'] = df['Close'].rolling(window=200).mean()
        df['Stock_Return'] = df['Close'].pct_change()

        combined = df.join(spy, how='inner')
        combined['Mansfield_RS'] = combined['Stock_Return'] - combined['SPY_Return']
        combined['MRS_52MA'] = combined['Mansfield_RS'].rolling(window=52).mean()

        if len(combined) < 200:
            print(f"⚠️  {ticker}: Not enough data")
            return {'Ticker': ticker, 'Buy': 'No Data'}

        last = combined.iloc[-1]
        last_close = float(last['Close'])
        last_50 = float(last['50MA'])
        last_200 = float(last['200MA'])
        last_rs = float(last['Mansfield_RS'])
        last_rs_ma = float(last['MRS_52MA'])

        # Distance from 200 MA
        above_200_pct = round((last_close - last_200) / last_200 * 100, 2)

        # Previous high & % drop
        high_price = float(combined['Close'].max())
        high_date = combined['Close'].idxmax()
        if isinstance(high_date, pd.Series):
            high_date = high_date.iloc[0]
        high_date = high_date.strftime('%Y-%m-%d')

        percent_from_high = round(100 * (high_price - last_close) / high_price, 2)
        if last_close > high_price:
            percent_from_high = -abs(percent_from_high)

        # Criteria for buy logic
        ma_rising = combined['50MA'].iloc[-3] < combined['50MA'].iloc[-2] < combined['50MA'].iloc[-1]
        above_200 = last_50 > last_200
        above_50 = last_close > last_50
        strong_rs = last_rs > last_rs_ma

        # Buy signal logic
        buy_signal = ''
        if abs(last_close - last_50) / last_50 <= 0.005:
            buy_signal = '💡 Opportunity to Buy'
        elif last_close < last_50:
            buy_signal = '❌ Sell'
        elif abs(last_close - last_50) / last_50 <= 0.02:
            buy_signal = '⚠️ Caution'
        elif above_200_pct > 100:
            buy_signal = '🚨 Overvalued'
        elif all([ma_rising, above_200, above_50, strong_rs]):
            buy_signal = '✅ Buy'

        return {
            'Ticker': ticker,
            'Price': round(last_close, 2),
            '50MA': round(last_50, 2),
            '200MA': round(last_200, 2),
            '%_Above_200MA': above_200_pct,
            'MRS': round(last_rs, 4),
            'MRS_52MA': round(last_rs_ma, 4),
            'Prev_High': round(high_price, 2),
            'High_Date': high_date,
            '%_From_High': percent_from_high,
            'Buy': buy_signal
        }

    except Exception as e:
        print(f"❌ Error processing {ticker}: {e}")
        return {'Ticker': ticker, 'Buy': 'No Data'}

# Analyze each stock
for symbol in stock_list:
    print(f"\n🔎 Analyzing {symbol}...")
    result = analyze_stock(symbol)
    if result:
        all_results.append(result)

# Display full report
if all_results:
    df = pd.DataFrame(all_results)
    print("\n📊 Full Stock Report:\n")
    print(tabulate(df, headers="keys", tablefmt="grid", showindex=False))
else:
    print("\n❌ No data available.")
